<h1> An Application of Athena Methods to Batch Comparison Audits </h1> 

This notebook illustrates a straightforward application of some of the methods described in <em>The Athena Class of Risk-Limiting Ballot Polling Audits</em> (Zagórski et al, https://arxiv.org/abs/2008.02315) to a scheme for batch comparison audits as in <em>Conservative Statistical Post-Election Audits</em> (Stark, https://projecteuclid.org/download/pdfview_1/euclid.aoas/1215118528). The code is exploratory, as it does not address multiple candidates, multiple contests, nor does it attempt to apply Athena to the more complex (and more efficient!) Kaplan-Markov method in <em>Risk Limiting Postelection Audits: Conservative </em>P<em>-values From Common Probability Inequalities</em> (Stark, https://www.stat.berkeley.edu/~stark/Preprints/pvalues09.pdf).

We begin with notation, which is largely borrowed from Stark's papers (above), but can be simplified because of our consideration of only the two-candidate case. An election has $N$ precincts (or batches), and has a reported margin $M$. Each precinct $p$ has a true (absolute) error $e_p$, which is intuitively understood as the reduction in the margin that occurs when rectifying the batch total. A value of $e_p = 0$ means the batch total was correct; $e_p < 0$ means the batch erred in the direction of favoring the reported winner; $e_p > 0$ means the batch erred in the direction of favoring the reported loser. Let $v_{pw}$ and $v_{pl}$ be the reported tallies in precinct $p$ for the reported winner and reported loser, respectively, and $a_{pw}$ and $a_{pl}$ be the true tallies in precinct $p$ for the reported winner and reported loser, respectively. Then $e_p$ = $(v_{pw} - v_{pl}) - (a_{pw} - a_{pl})$. Let $u_p$ be a hard upper bound for $e_p$. This may involve a physical element, for example, how many ballots were shipped to the precinct.

The precinct class contains all the necessary fields: a unique integer identifying the precinct, the upper bound error, and the true error.

In [32]:
from scipy.stats import hypergeom
from scipy.signal import fftconvolve
from typing import List, Dict

class Precinct():
    def __init__(self, intid: int, up: int, ep: int=-(2 ** 31)):
        self.intid = intid
        self.up = up
        self.ep = ep

The audit itself has a round schedule $n_1, n_2, \dots, n_j$ and a corresponding explicit risk schedule $\alpha_1, \alpha_2, \dots, \alpha_j$. The risk limit of the audit is (no larger than) $\sum_{i = 1}^j \alpha_i$.

In [33]:
class BatchComparisonAudit():
    def __init__(self, precincts: List[Precinct], margin: int, rounds: List[int], 
    risk_sched: List[float], tolerance: int):
        self.precincts = precincts
        self.margin = margin
        self.rounds = rounds
        self.tolerance = tolerance
        self.kmaxs = []
        self.risk_sched = risk_sched
        self.realized_risk_sched = []
        self.current_x = 0
        self.current_dist = []
        self.N = len(self.precincts)

The crux of the mathematics of the audit is a differentiation between "good" batches and "bad" batches, where "good" batches have $e_p \leq t$, where $t$ is a small tolerance and "bad" batches gave $e_p > t$. If enough batches are pulled and they are mostly or all "good", then there is evidence that there are few "bad" batches which could tip the margin in the reported loser's favor.

Risk-limting audits must devise and assume the worst-case election, that is, the election that is incorrectly called but most difficult to discern as incorrect. In ballot-polling audits, this generally means assuming the underlying tally is a tie, because that is hardest to differentiate from the reported margin (which will be $> .5$). In our case, it means assuming as few precincts as possible carry just enough error to close the margin. We denote by $x$ the fewest number of precincts that could flip the outcome: $x := |U_i|$, where $U_i$ is the smallest list of errors (true or upper bound) satisfying $\sum_{r \in U_i} r \geq M$, where each $r$ is either $e_p$ or $u_p$ for some precinct $p$. The computation of $x$ is somewhat complicated when we let each batch make a small "tolerance" error.

In [34]:
def compute_x(bca):
        # Sort the lists of upper bound and true errors, and initialize pointers and a running sum
        # to 0.
        upsort = sorted(bca.precincts, key=lambda p: p.up, reverse=True)
        epsort = sorted(bca.precincts, key=lambda p: p.ep, reverse=True)
        upptr = 0
        epptr = 0
        esum = 0
        
        # The only batches that are excluded from the assumption of making a tolerance error are
        # those which have been audited (and do in fact have an error < tolerance), and those which 
        # we are assuming of making an error > tolerance.
        less_than_tolerance_batches = 0
        less_than_tolerance_cumulative_error = 0
        for p in bca.precincts:
            if p.ep != -(2 ** 31) and p.ep < bca.tolerance:
                less_than_tolerance_batches += 1
                # Batches that have error
                less_than_tolerance_cumulative_error += p.ep

        # The worst x batches should exceed the margin minus an account for tolerance.
        while esum < bca.margin - less_than_tolerance_cumulative_error - (bca.N - less_than_tolerance_batches - (upptr + epptr)) * bca.tolerance:
            # If the largest unused upper bound error is larger than the largest unused true error,
            # advance the upper bound pointer. Otherwise, if there are no more upper bound errors 
            # left, or an unused true error is larger than an upper bound error, advance the true 
            # error pointer.
            upnext = upsort[upptr].up >= epsort[epptr].ep and upptr < len(upsort)
            if upnext:
                esum += upsort[upptr].up
                upptr += 1
            else:
                esum += epsort[epptr].ep
                epptr += 1

        bca.current_x = upptr + epptr
        return upptr + epptr # To see in Notebook

After a round is conducted, the true errors found (respective to the unique precinct intid's) are inputted via the following method. It is not strictly necessary to do so, but it may desirably increase $x$.

In [35]:
def record_round(bca, eplist: Dict[int, int]):
        for p in bca.precincts:
            if p.intid in eplist:
                p.ep = eplist[p.intid]

Given the computed value of $x$, which represents the least (worst-case) number of bad batches in the population, we can generate the distribution of the number of bad batches in the sample (for a given round). For the first round, this is straightforward: it is the hypergeometric distribution where the population size is N, the population "successes" (bad batches) is x, the sample size is $n_i$ for some $1 \leq i \leq j$, and the number of sample successes is $k$. For further rounds, it is a convolution which we elaborate on later.

In [36]:
def compute_dist(bca, rnd_i):
        if rnd_i == 0:
            bca.current_dist = hypergeom.pmf(range(bca.rounds[0] + 1), bca.N, bca.current_x, 
                bca.rounds[0])
        else:
            # We avoid having to use a complicated manual convolution because the third parameter, x,
            # does not change mid-round (or rather if it does, all past rounds are recomputed).
            sample_good_batches = range(bca.rounds[rnd_i] - bca.rounds[rnd_i - 1] + 1)
            unsampled_N = bca.N - bca.rounds[rnd_i - 1]
            sample_size = bca.rounds[rnd_i] - bca.rounds[rnd_i - 1] + 1
            bca.current_dist = fftconvolve(bca.current_dist, 
                hypergeom.pmf(sample_good_batches, unsampled_N, bca.current_x, sample_size))

Naturally, lower values of $k$ are more desirable, and we can employ the notion of a $k_{max}$. The $k_{max}(n_i)$ is the largest value of $k$ in round $n_i$ such that the left tail up to $k$ (inclusive) is less than $\alpha_i$ (i.e., the audit can stop). Rounds may not have a $k_{max}$ if the sample size is too small. We find the $k_{max}$ with a linear search.

In [37]:
def find_kmax(bca, rnd_i):
        tail = 0
        for i in range(len(bca.current_dist) - 1):
            tail += bca.current_dist[i]
            if tail < bca.risk_sched[rnd_i] and tail + bca.current_dist[i+1] >= bca.risk_sched[rnd_i]:
                bca.kmaxs.append(i)
                return
        bca.kmaxs.append(None)

Mirroring the form of Athena class audits, we now observe that values of $k \leq k_{max}$ do not proceed to further rounds, and so can be truncated from the probability distribution. Unlike in Athena audits, where a right tail is truncated, here a left tail is truncated because sufficiently low values, not high ones, let the audit stop. This truncated distribution gets convolved with a "fresh" hypergeometric distribution to produce the next round's distribution.

In [38]:
def truncate_dist(bca):
        if bca.kmaxs[-1] is None:
            bca.realized_risk_sched.append(0)
            return
        bca.realized_risk_sched.append(sum(bca.current_dist[:bca.kmaxs[-1] + 1]))
        # True truncation is unacceptable since the distribution does not actually start at k=0 after truncation.
        for i in range(bca.kmaxs[-1] + 1):
            bca.current_dist[i] = 0

The last three methods are combined into a function that computes the audit up to a given round, and based on the current $x$. Such functionality is useful because, if $x$ increases mid-audit (which may happen if some $p$ with large $u_p$ are found to have small $e_p$), then we know with certainty a past assumption was too conservative, and we can recompute the entire audit thus far, permitting auditors to retroactively stop if they satisfy any of the recomputed $k_{max}$.

In [39]:
def compute_rounds(bca, i):
        bca.kmaxs = []
        bca.realized_risk_sched = []
        # The audit computation procedure mirrors that of Minerva, except in that the audit is
        # scheduled beforehand.
        for rnd in range (i+1):
            compute_dist(bca, rnd)
            find_kmax(bca, rnd)
            truncate_dist(bca)

<h2> Audit Scenario 1 </h2>

Consider an election with $N = 1,000$ precincts, each with $1,000$ ballots ($510$ for the reported winner and $490$ for the reported loser) is audited with a single-round of $100$ and a risk limit of $.2$.

For the hard error upper bounds, assume that each precinct was delivered $1,100$ ballots (and so $100$ ballots per precinct went unused or were counted as invalid). Then the upper bound (corresponding to the worst-case possibility that all $1,100$ ballots really were cast for the reported loser) is $u_p = 1,100 + (510 - 490) = 1,120$. That is, correcting the precinct could "close the margin" by up to $1,120$ votes, given our assumption of $1,100$ delivered ballots.

If the election was completely correctly called (that is, every hand precinct count reveals the reported count) the audit will stop at the first point there is a $k_{max}$. (So a multiple-round schedule with these assumptions is rather disingenuous since exactly all rounds except possibly one will have stopping probabilities and risks of $0$%, the exception having stopping probability $100$%.)

Assume a tolerance of $t = 0$.

We begin by constructing the necessary audit objects.

In [40]:
N = 1000
margin = 20000
plist = []
for i in range(N):
    plist.append(Precinct(i, 1120))
bca = BatchComparisonAudit(plist, margin, [100], [.2], 0)

We compute $x$, and then the first and only round of this audit.

In [41]:
compute_x(bca)

18

In [42]:
compute_rounds(bca, 0)

From the computation we can glean the $k_{max}$ and the precise <em>p</em>-value associated with the $k_{max}$.

In [43]:
bca.kmaxs

[0]

In [44]:
bca.realized_risk_sched

[0.147533268723528]

Even if no errors are found, one tenth of the batches must be audited to attain a risk of $14.75$%!

We see that the current distribution's value at $k = 0$ is zeroed out due to the truncation.

In [45]:
bca.current_dist

array([0.00000000e+00, 3.00747320e-01, 2.86288314e-01, 1.69077242e-01,
       6.94151769e-02, 2.10358507e-02, 4.87599016e-03, 8.83836879e-04,
       1.26989484e-04, 1.45691889e-05, 1.33768674e-06, 9.80489107e-08,
       5.69392910e-09, 2.58392313e-10, 8.96051149e-12, 2.29090744e-13,
       4.06585047e-15, 4.46942930e-17, 2.28989279e-19, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
      

^ this is hypergeometric(k, N, x, n)

<h2> Audit Scenario 2 </h2>

Consider the same election parameters as in Audit Scenario 1, but now the round schedule is $[100, 200]$ and the risk schedule is $[.15, .05]$ (giving $\alpha = .20$).

In [51]:
N = 1000
margin = 20000
plist = []
for i in range(N):
    plist.append(Precinct(i, 1120))
bca = BatchComparisonAudit(plist, margin, [100, 200], [.15, .05], 0)

In [52]:
compute_x(bca)

18

In [53]:
compute_rounds(bca, 1)

In [54]:
bca.kmaxs

[0, 1]

In [55]:
bca.realized_risk_sched

[0.147533268723528, 0.034531485223801874]

And so we can get one "bad" batch and still stop with this round-risk schedule.

<h2> Audit Scenario 3 </h2>

Now let's consider a nonzero tolerance. Let's say we have a round size of $200$ (and risk limit of $.25$). Suppose the election is as in the earlier scenarios, except that instead of each precinct having 0 true error, true error is distributed among the sampled batches in the following way:

- 180 of the 200 batches have 0 true error
- 10 of the 200 batches have 1 true error
- 5 of the 200 batches have 2 true error
- 3 of the 200 batches have 3 true error
- 1 of the 200 batches has 4 true error
- 1 of the 200 batches has 15 true error

What is the risk computation if we select a tolerance of 4? There is a tradeoff involved: assuming a larger tolerance t may decrease the value of $x$ (which is undesirable, because the margin-closing errors are assumed to be concentrated in fewer precincts), but desirably decreases $k$, our interpretation of how many batches are “bad.” For a $t = 4$, we see that a $k_{max}$ of $0$ will not let us stop (1 batch exceeds the tolerance).

In [61]:
N = 1000
margin = 20000
plist = []
for i in range(N):
    plist.append(Precinct(i, 1120))
bca = BatchComparisonAudit(plist, margin, [200], [.2], 4)
    
# Some true errors, so that the tolerance is useful.
eplist = {}
for i in range(180):
    eplist[i] = 0
for i in range(10):
    eplist[i+180] = 1
for i in range(5):
    eplist[i+190] = 2
for i in range(3):
    eplist[i+195] = 3
    eplist[198] = 4
    eplist[199] = 15
record_round(bca, eplist)

Note that the value of $x$ has decreased from when we chose $t = 0$:

In [62]:
compute_x(bca)

16

In [63]:
compute_rounds(bca, 0)

In [64]:
bca.kmaxs

[1]

In [65]:
bca.realized_risk_sched

[0.13861689005259878]

With the assumed errors and a tolerance choice of $t = 4$, we can stop the audit with a <em>p</em>-value of .1386.

What if we had chosen $t = 15$? This would likely decrease $x$ even further, but, in exchange, a $k_{max}$ of 0 would let us stop.

In [66]:
N = 1000
margin = 20000
plist = []
for i in range(N):
    plist.append(Precinct(i, 1120))
bca = BatchComparisonAudit(plist, margin, [200], [.5], 15)
    
# Some true errors, so that the tolerance is useful.
eplist = {}
for i in range(180):
    eplist[i] = 0
for i in range(10):
    eplist[i+180] = 1
for i in range(5):
    eplist[i+190] = 2
for i in range(3):
    eplist[i+195] = 3
    eplist[198] = 4
    eplist[199] = 15
record_round(bca, eplist)

Just to clarify the calculation of $x$ now that we must account for large tolerances: The lowest value of $x$ such that $20000 - (180 \cdot 0 + 10 \cdot 1 + 5 \cdot 2 + 3 \cdot 3 + 1 \cdot 4 + 1 \cdot 15) - ((800 - x) \cdot 15) - (x \cdot 1120)$ is $\leq 0$ is 8. The value of that expression for $x = 7$ is $217$, which means that six terrible precincts cannot flip the election.

In [68]:
compute_x(bca)

8

In [69]:
compute_rounds(bca, 0)

In [70]:
bca.kmaxs

[0]

In [71]:
bca.realized_risk_sched

[0.1665952541260108]

And we see that we have a lower risk by assuming the smaller tolerance of $t = 4$!